In [23]:
import csv

with open('game_dic.tsv', 'w', encoding='utf-8', newline='') as f:
  tw = csv.writer(f, delimiter='\t')
  tw.writerow(['길마', 'guild master'])
  tw.writerow(['gg', 'good game'])
  tw.writerow(['지지', 'good game'])
  tw.writerow(['용계', 'valley of dragon'])

In [24]:
!gsutil cp game_dic.tsv gs://jins62/game_dic.tsv
!gsutil ls gs://jins62

Copying file://game_dic.tsv [Content-Type=text/tab-separated-values]...
-
Operation completed over 1 objects/78.0 B.                                       
gs://jins62/2024-06-12T05:38:10+00:00_76700a
gs://jins62/2024-06-12T05:38:11+00:00_ddcb5b
gs://jins62/2024-06-12T05:38:15+00:00_4cd6e5
gs://jins62/2024-06-12T05:38:16+00:00_df6d07
gs://jins62/README.ipynb
gs://jins62/game_dic.tsv
gs://jins62/generated_workspace_file.json
gs://jins62/gws_log - activity.csv
gs://jins62/poc.jpg
gs://jins62/ai-platform/
gs://jins62/audio-files/
gs://jins62/eager/
gs://jins62/estimators/
gs://jins62/explainable_ai/
gs://jins62/fast-and-lean-data-science/
gs://jins62/images/
gs://jins62/keras/
gs://jins62/load_data/
gs://jins62/machine_learning_deepdive/
gs://jins62/python-docs-samples/
gs://jins62/representation/
gs://jins62/sequences/
gs://jins62/tf2_course/
gs://jins62/transcripts/
gs://jins62/vertex_samples/


In [25]:
from google.cloud import translate

client = translate.TranslationServiceClient()

In [26]:
PROJECT_ID=!gcloud config get-value project
PROJECT_ID=PROJECT_ID[0]
LOCATION="us-central1"

In [27]:
def translate_text(
    txt: str,
    project_id: str = PROJECT_ID,
    location: str = LOCATION,
    src_lang: str = 'ko',
    dest_lang: str = 'en',
    glossary: str | None = None
) -> translate.TranslateTextResponse:
  parent = client.common_location_path(project_id, location)
  glossary_config = (
      translate.TranslateTextGlossaryConfig(glossary=glossary)
      if glossary
      else None
  )
  request = translate.TranslateTextRequest(
      parent=parent,
      contents=[txt],
      source_language_code=src_lang,
      target_language_code=dest_lang,
      glossary_config=glossary_config
  )
  return client.translate_text(request = request)

In [33]:
def create_glossary(
    input_uri: str,
    glossary_id: str,
    project_id: str = PROJECT_ID,
    location: str = LOCATION,
    timeout: int = 180,
    src_lang: str = 'ko',
    dest_lang: str = 'en'
)->translate.Glossary:
  name = client.glossary_path(project_id, location, glossary_id)
  language_pair = translate.Glossary.LanguageCodePair(
      source_language_code=src_lang,
      target_language_code=dest_lang
  )
  gcs_source = translate.GcsSource(input_uri=input_uri)
  input_config = translate.GlossaryInputConfig(gcs_source=gcs_source)
  # request = translate.CreateGlossaryRequest(input_config=input_config)

  operation = client.delete_glossary(name=name)
  glossary = translate.Glossary(
      name = name,
      language_pair = language_pair,
      input_config = input_config
  )

  parent = client.common_location_path(project_id, location)
  operation = client.create_glossary(
      parent=parent,
      glossary=glossary,
      timeout=timeout
  )
  return operation.result()

In [34]:
glossary = create_glossary(
    input_uri="gs://jins62/game_dic.tsv",
    glossary_id="game_dic")

print(glossary)

name: "projects/429718924597/locations/us-central1/glossaries/game_dic"
language_pair {
  source_language_code: "ko"
  target_language_code: "en"
}
input_config {
  gcs_source {
    input_uri: "gs://jins62/game_dic.tsv"
  }
}
entry_count: 4
submit_time {
  seconds: 1737103635
  nanos: 790167894
}
end_time {
}
display_name: "game_dic"



In [30]:
def get_src_lang(sentence):
  parent = client.common_location_path(PROJECT_ID, LOCATION)
  response = client.detect_language(
      parent=parent,
      content=sentence
  )
  return response.languages[0]

In [31]:
txt = "길마가 용계 입구로 모이시랍니다."
src_lang = get_src_lang(txt)
print(src_lang)

language_code: "ko"
confidence: 1



In [35]:
response = translate_text(txt=txt, src_lang=src_lang.language_code, glossary=glossary.name)

# Display the translation for each input text provided
for translation in response.translations:
    print(f"Default Translated text: {translation.translated_text}")

for translation in response.glossary_translations:
    print(f"Glossary Translated text: {translation.translated_text}")

Default Translated text: Gilma gathers at the entrance to Yonggye.
Glossary Translated text: guild master, please gather at the entrance to valley of dragon .
